In [25]:
import tweepy
import json
import yaml
import pandas as pd
import pymongo
import nltk
import re

In [2]:
#Specify how many pages of tweets we are importing (each page is 200 tweets)
pages_to_query = 3
q1_words = 100
q2_words = 20
q3_words = 50

#login credential
config = {
    'CONSUMER_KEY': 'EFoNIPpaC8GeL3qBWGxtkmTxJ',
    'CONSUMER_SECRET': 'O3xwQBGcP03afTJ4dKJkKmDhylJIMqXGsRQ2Y6fSQskqp3OmyB',
    'OAUTH_SECRET': 'XTrufn3hzMduJAmO5Z4rbXTOC8azY7T9OPluFrOjJNo5h',
    'OAUTH_TOKEN': '1112509961865699329-ByU2qCjiC85vHPOhwrrlfCXN5zTTGq'
}

In [26]:
# function that takes a word and returns true if it consists only of non-alphabetic characters
def alpha_filter(w):
    # pattern to match a word of non-alphabetical characters
    pattern = re.compile('^[^a-z]+$')
    if (pattern.match(w)):
        return True
    else:
        return False 

#function that performs NLP analysis on a list of strings
def nlp_analysis(listOfStrings, numOfWords):
    #tokenization
    all_tokens = [tok.lower() for string in listOfStrings for tok in nltk.word_tokenize(string)]
    #remove stopwords
    nltk_stopwords = nltk.corpus.stopwords.words('english')
    all_stopped_tokens = [tok for tok in all_tokens if not tok in nltk_stopwords]
    token_list = [tok for tok in all_stopped_tokens if not alpha_filter(tok)]
    #Final step, count the frequency of the tokens and interpret the meaning.
    msgFD = nltk.FreqDist(token_list)
    top_words = msgFD.most_common(numOfWords)
    return top_words

In [3]:
#creating a database
client = pymongo.MongoClient()
client.database_names()

#create a new db called cyber_tweets
db = client.cyber_db

#create a new collection(a group of documents) called deltatweets
cyber_timeline = db.cyber_tweets
cyber_timeline.drop() # drop all existing data, just to refresh the table as new tweets comes in in the future

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#login to twitter API
auth = tweepy.OAuthHandler(config['CONSUMER_KEY'],config['CONSUMER_SECRET'])
auth.set_access_token(config['OAUTH_TOKEN'],config['OAUTH_SECRET'])
api = tweepy.API(auth)

In [5]:
#insert 4 pages worth of tweets into cyber_tweets collection

for i in range(1, pages_to_query+1):
    #use API to scrape CyberpunkGame's tweets and all information related to the tweets
    timeline = api.user_timeline('CyberpunkGame', count=200, page=i)

    #convert the data(tweepy.models.ResultSet) into a python list of tweets
    tweet_list = list(timeline)

    #use list comprehension to extract json field of the tweets
    tweet_json = [tweet._json for tweet in tweet_list]

    #store json tweets into cyber_db's cyber_tweets collection
    cyber_timeline.insert_many(tweet_json)

In [6]:
#query data from cyber_db
docs = cyber_timeline.find()
#docs[123]

In [7]:
#Question 1: Top 100 most favorited tweets NLP analysis.
cyber_df1 = pd.DataFrame(list(docs)) #WARNING: this line cannot be run more that once.
cyber_df1_sorted = cyber_df1.sort_values(by=['favorite_count'], ascending=False) #sort by descending favorite_count

In [20]:
cyber_top100fav = cyber_df1_sorted.to_dict('records')[:100] #convert pd dataframe back to a list of dictionaries
cyber_top100fav[:1]

[{'_id': ObjectId('5cbd1261c5cd333b69daea73'),
  'contributors': None,
  'coordinates': None,
  'created_at': 'Thu Feb 28 10:39:01 +0000 2019',
  'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
  'extended_entities': nan,
  'favorite_count': 63806,
  'favorited': False,
  'geo': None,
  'id': 1101069253590032385,
  'id_str': '1101069253590032385',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': nan,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': nan,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'place': None,
  'possibly_sensitive': nan,
  'quoted_status': nan,
  'quoted_status_id': nan,
  'quoted_status_id_str': nan,
  'retweet_count': 10514,
  'retweeted': False,
  'retweeted_status': nan,
  'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
  'text': 'For those of you asking, yes, we will be at E3 this year.',
  'truncated': False,
  'user': {'id': 821102114,
 

In [22]:
#Takes the top 100 tweet's 'text' field and store into a list of strings.
cyber_top100_texts = [tweet['text'] for tweet in cyber_top100fav if 'text' in tweet.keys()]
cyber_top100_texts[:2]

['For those of you asking, yes, we will be at E3 this year.',
 '&lt;3 https://t.co/OJpwfgJcg4']

In [27]:
#Top 100 favorite tweet's top 30 words by frequency
nlp_analysis(cyber_top100_texts, q1_words)

[('https', 50),
 ("n't", 8),
 ('cyberpunk2077', 6),
 ('cdprojektred', 6),
 ('say', 6),
 ('que', 6),
 ('porsche', 5),
 ('alexwolfcosplay', 5),
 ('year', 4),
 ('good', 4),
 ('djhyper', 4),
 ('la', 4),
 ('gronkh', 4),
 ("'s", 4),
 ('yes', 3),
 ('lt', 3),
 ('happy', 3),
 ('anaeart', 3),
 ('course', 3),
 ('game', 3),
 ('gt', 3),
 ('v', 3),
 ('cyberpunk', 3),
 ('ready', 3),
 ('us', 3),
 ('know', 3),
 ('would', 3),
 ('es', 3),
 ('please', 3),
 ('um', 3)]